# BGer Scraper - Code-Erklärung für Anfänger

In diesem Notebook erkläre ich dir Schritt für Schritt, wie der Web Scraper funktioniert, den wir für die Bundesgerichts-Regesten geschrieben haben.

## Was ist Web Scraping?

**Web Scraping** bedeutet, automatisiert Daten von Websites zu extrahieren. Statt manuell jede Seite zu besuchen und den Text zu kopieren, schreibt man ein Programm, das dies automatisch macht.

**Unser Ziel:** Alle Regesten aus BGE 151 III automatisch in eine CSV-Datei exportieren.

---
# Teil 1: HTML-Grundlagen

Bevor wir den Code verstehen können, müssen wir verstehen, wie Websites aufgebaut sind.

## Was ist HTML?

**HTML** (HyperText Markup Language) ist die Sprache, in der Websites geschrieben sind. Es besteht aus **Tags**, die den Inhalt strukturieren.

### Beispiel einer einfachen HTML-Struktur:

In [ ]:
# Dies ist KEIN Python-Code, sondern zeigt dir, wie HTML aussieht:

beispiel_html = """
<html>

  <head>
    <title>Meine Webseite</title>
  </head>

  <body>
    <h1>Willkommen</h1>
    <p>Dies ist ein Absatz.</p>
    <div class="wichtig">
      <span>Text in einem Span</span>
    </div>
  </body>

</html>
"""

print("So sieht HTML aus:")
print(beispiel_html)

### Die wichtigsten HTML-Begriffe:

| Begriff | Erklärung | Beispiel |
|---------|-----------|----------|
| **Tag** | Ein HTML-Element, eingeschlossen in `< >` | `<p>`, `<div>`, `<span>` |
| **Öffnender Tag** | Beginnt ein Element | `<div>` |
| **Schliessender Tag** | Beendet ein Element (mit `/`) | `</div>` |
| **Attribut** | Zusätzliche Information im Tag | `class="wichtig"`, `id="regeste"` |
| **class** | Eine Kategorie/Gruppe (kann mehrfach vorkommen) | `<div class="big bold">` |
| **id** | Ein eindeutiger Bezeichner | `<div id="regeste">` |

### Häufige HTML-Tags:

- `<div>` - Ein Block-Container (wie eine Box)
- `<span>` - Ein Inline-Container (für Text innerhalb einer Zeile)
- `<p>` - Ein Absatz (Paragraph)
- `<a href="...">` - Ein Link
- `<h1>`, `<h2>`, etc. - Überschriften

## Die HTML-Struktur der BGer-Website

Um einen Scraper zu schreiben, muss man zuerst die HTML-Struktur der Website analysieren. 

**So habe ich das gemacht:**
1. Website im Browser öffnen
2. Rechtsklick → "Untersuchen" (oder F12 für Entwicklertools)
3. Die HTML-Struktur anschauen und Muster erkennen

### Was ich auf der BGer-Website gefunden habe:

Jede Regeste ist in einem `<div>` mit `id="regeste"` eingeschlossen:

In [ ]:
# So sieht die HTML-Struktur einer Regeste auf bger.ch aus:

bger_html_beispiel = """
<div id="regeste" lang="de">
  <div class="big bold">Regeste</div>
  <br>
  <div class="paraatf">
    <span class="artref">Art. 125 ZGB</span>; nachehelicher Unterhalt bei Scheidung.
    <div class="paratf">
      Die Rechtsprechung, wonach die Pflicht zur Leistung nachehelichen 
      Unterhaltes im Grundsatz längstens bis zum Erreichen des ordentlichen 
      Pensionierungsalters dauert...
    </div>
  </div>
</div>
"""

print("HTML-Struktur einer BGer-Regeste:")
print(bger_html_beispiel)

### Analyse der Struktur:

```
<div id="regeste">           ← Container für die gesamte Regeste
  │
  ├── <div class="big bold">  ← Das Label ("Regeste" oder "Regeste a")
  │
  └── <div class="paraatf">   ← Der Inhalt
        │
        ├── <span class="artref">  ← Gesetzesartikel
        │
        └── <div class="paratf">   ← Weiterer Text
```

**Wichtige Erkenntnis:** Bei Entscheiden mit mehreren Regesten gibt es mehrere `<div id="regeste">` Blöcke!

In [ ]:
# Bei mehreren Regesten sieht es so aus:

mehrere_regesten_html = """
<div id="regeste" lang="de">
  <div class="big bold">Regeste a</div>
  <div class="paraatf">
    <span class="artref">Art. 51 Abs. 4 BGG</span>; Streitwert...
  </div>
</div>

<div id="regeste" lang="de">
  <div class="big bold">Regeste b</div>
  <div class="paraatf">
    <span class="artref">Art. 84 SchKG</span>; Kognition...
  </div>
</div>
"""

print("HTML bei mehreren Regesten (z.B. BGE 151 III 45):")
print(mehrere_regesten_html)

---
# Teil 2: Das Kerngerüst des Scrapers

Jetzt schauen wir uns den eigentlichen Python-Code an.

## Der Ablauf in 4 Schritten:

```
┌─────────────────────────────────────────────────────────────────┐
│  1. INDEX-SEITE LADEN                                          │
│     → Liste aller BGE 151 III Entscheide holen                 │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  2. LINKS EXTRAHIEREN                                          │
│     → URLs zu allen Einzelentscheiden sammeln                  │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  3. JEDEN ENTSCHEID BESUCHEN                                   │
│     → Urteilsnummer und Regeste extrahieren                    │
└─────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────┐
│  4. CSV SCHREIBEN                                              │
│     → Alle Daten in eine Datei speichern                       │
└─────────────────────────────────────────────────────────────────┘
```

## Die Imports - Was brauchen wir?

Schauen wir uns zuerst die benötigten Bibliotheken an:

In [ ]:
# Diese Bibliotheken werden importiert:

import csv                      # Zum Schreiben von CSV-Dateien
import re                       # Regular Expressions (Textmuster suchen)
import time                     # Zum Warten zwischen Anfragen
from urllib.parse import urljoin  # URLs zusammenbauen

import requests                 # HTTP-Anfragen an Websites senden
from bs4 import BeautifulSoup   # HTML parsen ("lesen und verstehen")

print("Alle Bibliotheken erfolgreich importiert!")

### Was macht jede Bibliothek?

| Bibliothek | Zweck | Beispiel |
|------------|-------|----------|
| `csv` | CSV-Dateien lesen/schreiben | `csv.writer(file)` |
| `re` | Textmuster suchen (Regular Expressions) | `re.search(r"BGE 151", text)` |
| `time` | Pausen einlegen | `time.sleep(0.5)` |
| `requests` | Websites herunterladen | `requests.get(url)` |
| `BeautifulSoup` | HTML analysieren | `soup.find("div", id="regeste")` |

---
# Teil 3: Die Funktionen im Detail

## Funktion 1: `fetch_html()` - Eine Webseite herunterladen

In [ ]:
# Konfiguration: Wie wir uns gegenüber der Website "vorstellen"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0 Safari/537.36",
    "Accept-Language": "de-CH,de;q=0.9,en;q=0.6",
}

# Was bedeutet das?
# - User-Agent: Sagt der Website, welchen Browser wir "simulieren"
# - Accept-Language: Wir möchten deutsche Inhalte

print("Headers konfiguriert:")
for key, value in HEADERS.items():
    print(f"  {key}: {value[:50]}...")

In [ ]:
def fetch_html(session, url):
    """
    Lädt den HTML-Inhalt einer URL herunter.
    
    Parameter:
    - session: Eine requests.Session (hält Verbindungen offen)
    - url: Die Adresse der Webseite
    
    Rückgabe:
    - Der HTML-Text der Seite als String
    """
    # GET-Anfrage an die URL senden
    r = session.get(url, headers=HEADERS, timeout=30, allow_redirects=True)
    
    # Prüfen, ob die Anfrage erfolgreich war (Status 200 = OK)
    r.raise_for_status()
    
    # Encoding sicherstellen (für Umlaute ä, ö, ü)
    if not r.encoding:
        r.encoding = r.apparent_encoding or "utf-8"
    
    return r.text

print("Funktion fetch_html() definiert.")
print("")
print("Was passiert hier?")
print("1. session.get(url) → Sendet eine Anfrage an die Website")
print("2. raise_for_status() → Wirft einen Fehler, wenn etwas schiefgeht")
print("3. r.text → Gibt den HTML-Inhalt zurück")

In [ ]:
# Beispiel: Eine Seite herunterladen

with requests.Session() as session:
    url = "https://search.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?highlight_docid=atf%3A%2F%2F151-III-1%3Ade&lang=de&zoom=&type=show_document"
    html = fetch_html(session, url)
    
    print(f"HTML heruntergeladen!")
    print(f"Länge: {len(html)} Zeichen")
    print(f"")
    print(f"Die ersten 500 Zeichen:")
    print(html[:500])

## Funktion 2: `normalize_ws()` - Text aufräumen

In [ ]:
def normalize_ws(s):
    """
    Räumt Whitespace (Leerzeichen, Tabs, Zeilenumbrüche) auf.
    
    - Mehrere Leerzeichen/Tabs → ein Leerzeichen
    - Mehr als 2 Zeilenumbrüche → maximal 2
    - Leerzeichen am Anfang/Ende entfernen
    """
    # Mehrere Leerzeichen/Tabs durch ein Leerzeichen ersetzen
    s = re.sub(r"[ \t]+", " ", s)
    
    # Mehr als 2 Zeilenumbrüche durch 2 ersetzen
    s = re.sub(r"\n{3,}", "\n\n", s)
    
    # Leerzeichen am Anfang und Ende entfernen
    return s.strip()

# Beispiel:
unordentlicher_text = "   Viel    zu   viele     Leerzeichen   \n\n\n\n\n und Zeilenumbrüche   "
sauberer_text = normalize_ws(unordentlicher_text)

print("Vorher:")
print(repr(unordentlicher_text))
print("")
print("Nachher:")
print(repr(sauberer_text))

### Exkurs: Regular Expressions (Regex)

Regular Expressions sind Muster, um Text zu suchen/ersetzen:

| Muster | Bedeutung |
|--------|----------|
| `[ \t]+` | Ein oder mehr Leerzeichen oder Tabs |
| `\n{3,}` | Drei oder mehr Zeilenumbrüche |
| `\b` | Wortgrenze |
| `[0-9]+` | Eine oder mehr Ziffern |
| `.*` | Beliebig viele beliebige Zeichen |

## Funktion 3: `extract_decision_links()` - Links zu Entscheiden finden

In [ ]:
def extract_decision_links(index_html, base_url):
    """
    Extrahiert alle Links zu Einzelentscheiden aus der Index-Seite.
    
    Parameter:
    - index_html: Der HTML-Text der Index-Seite
    - base_url: Die Basis-URL für relative Links
    
    Rückgabe:
    - Liste von URLs zu den einzelnen Entscheiden
    """
    # HTML mit BeautifulSoup parsen ("verstehen")
    soup = BeautifulSoup(index_html, "lxml")
    
    links = []
    
    # Alle <a>-Tags (Links) finden, die ein href-Attribut haben
    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        
        # Relative URLs zu absoluten machen
        abs_url = urljoin(base_url, href)
        
        # Nur Links zu Entscheiden behalten
        # Diese erkennt man an "type=show_document" und "highlight_docid="
        if "type=show_document" in abs_url and "highlight_docid=" in abs_url:
            links.append(abs_url)
    
    # Duplikate entfernen (Reihenfolge behalten)
    seen = set()
    out = []
    for u in links:
        if u not in seen:
            seen.add(u)
            out.append(u)
    
    return out

print("Funktion extract_decision_links() definiert.")

In [ ]:
# Beispiel: Links von der Index-Seite extrahieren

INDEX_URL = "https://search.bger.ch/ext/eurospider/live/de/php/clir/http/index_atf.php?year=151&volume=III&lang=de&zoom=&system=clir"

with requests.Session() as session:
    index_html = fetch_html(session, INDEX_URL)
    links = extract_decision_links(index_html, INDEX_URL)
    
    print(f"Anzahl gefundener Entscheide: {len(links)}")
    print("")
    print("Die ersten 3 Links:")
    for i, link in enumerate(links[:3], 1):
        print(f"{i}. {link[:80]}...")

### Wie funktioniert BeautifulSoup?

BeautifulSoup verwandelt HTML-Text in ein Objekt, das wir durchsuchen können:

In [ ]:
# Beispiel: BeautifulSoup verwenden

beispiel_html = """
<html>
  <body>
    <div id="regeste">
      <div class="big bold">Regeste</div>
      <p>Dies ist der Inhalt.</p>
    </div>
    <a href="/seite1">Link 1</a>
    <a href="/seite2">Link 2</a>
  </body>
</html>
"""

soup = BeautifulSoup(beispiel_html, "lxml")

# Element mit bestimmter ID finden
regeste_div = soup.find("div", id="regeste")
print("Element mit id='regeste':")
print(regeste_div)
print("")

# Alle Links finden
alle_links = soup.find_all("a")
print("Alle Links:")
for link in alle_links:
    print(f"  - {link['href']} → '{link.text}'")
print("")

# Text aus einem Element extrahieren
text = regeste_div.get_text(" ", strip=True)
print(f"Text im regeste-div: '{text}'")

## Funktion 4: `parse_urteilsnummer()` - Die BGE-Nummer finden

In [ ]:
def parse_urteilsnummer(soup):
    """
    Extrahiert die Urteilsnummer (z.B. "151 III 45") aus der Seite.
    
    Verwendet einen Regular Expression, um das Muster zu finden.
    """
    # Gesamten Text der Seite holen
    full_text = soup.get_text("\n", strip=True)
    full_text = normalize_ws(full_text)
    
    # Muster suchen: z.B. "151 III 45" oder "BGE 151 III 45"
    # \b = Wortgrenze
    # (1[0-9]{2}) = Band-Nummer (100-199)
    # ([IVX]+) = Römische Ziffern (I, II, III, IV, V)
    # ([0-9]{1,4}) = Seitenzahl (1-4 Ziffern)
    m = re.search(r"\b(?:BGE\s*)?(1[0-9]{2})\s+([IVX]+)\s+([0-9]{1,4})\b", full_text)
    
    if not m:
        return ""
    
    # Gefundene Gruppen zusammensetzen
    return f"{m.group(1)} {m.group(2)} {m.group(3)}"

print("Funktion parse_urteilsnummer() definiert.")
print("")
print("Der Regex erklärt:")
print("  \\b(?:BGE\\s*)?(1[0-9]{2})\\s+([IVX]+)\\s+([0-9]{1,4})\\b")
print("  │    │          │            │           │")
print("  │    │          │            │           └── Seitenzahl (z.B. 45)")
print("  │    │          │            └── Röm. Ziffern (z.B. III)")
print("  │    │          └── Band (z.B. 151)")
print("  │    └── Optionales 'BGE '")
print("  └── Wortgrenze")

In [ ]:
# Beispiel: Urteilsnummer extrahieren

test_texte = [
    "BGE 151 III 45 ist ein wichtiger Entscheid",
    "Siehe auch 151 III 1 und 151 III 9",
    "Hier steht keine Nummer",
]

for text in test_texte:
    m = re.search(r"\b(?:BGE\s*)?(1[0-9]{2})\s+([IVX]+)\s+([0-9]{1,4})\b", text)
    if m:
        nummer = f"{m.group(1)} {m.group(2)} {m.group(3)}"
        print(f"'{text}' → Gefunden: {nummer}")
    else:
        print(f"'{text}' → Nichts gefunden")

## Funktion 5: `parse_regeste()` - Die Regeste extrahieren

**Dies ist die wichtigste Funktion!** Hier nutzen wir unser Wissen über die HTML-Struktur.

In [ ]:
def parse_regeste(soup):
    """
    Extrahiert alle Regesten aus der Seite.
    
    Die HTML-Struktur ist:
    <div id="regeste" lang="de">
      <div class="big bold">Regeste</div>  (oder "Regeste a", "Regeste b" etc.)
      <div class="paraatf">
        <span class="artref">Art. XY</span>; Titel...
        <div class="paratf">Eigentlicher Text...</div>
      </div>
    </div>
    
    Bei mehreren Regesten wird "Regeste a:", "Regeste b:" etc. vorangestellt.
    Bei nur einer Regeste wird kein Label verwendet.
    """
    
    # SCHRITT 1: Alle <div id="regeste"> finden
    regeste_divs = soup.find_all("div", id="regeste")
    
    if not regeste_divs:
        return ""  # Keine Regeste gefunden
    
    regesten = []
    
    # SCHRITT 2: Jede Regeste verarbeiten
    for div in regeste_divs:
        # Label extrahieren (z.B. "Regeste" oder "Regeste a")
        label_div = div.find("div", class_="big bold")
        label = label_div.get_text(strip=True) if label_div else ""
        
        # Inhalt extrahieren aus <div class="paraatf">
        content_div = div.find("div", class_="paraatf")
        if not content_div:
            continue
        
        # Text sauber extrahieren
        content = content_div.get_text(" ", strip=True)
        content = normalize_ws(content)
        
        if content:
            regesten.append((label, content))
    
    if not regesten:
        return ""
    
    # SCHRITT 3: Formatierung
    # Bei nur einer Regeste: kein Label
    if len(regesten) == 1:
        return regesten[0][1]  # Nur den Inhalt zurückgeben
    
    # Bei mehreren Regesten: mit Label
    parts = []
    for label, content in regesten:
        # Label normalisieren (geschütztes Leerzeichen entfernen)
        label_clean = label.replace("\u00a0", " ").strip()
        parts.append(f"{label_clean}:\n{content}")
    
    return "\n\n".join(parts)

print("Funktion parse_regeste() definiert.")

In [ ]:
# Beispiel: Regeste aus echtem HTML extrahieren

test_html = """
<html>
<body>
  <div id="regeste" lang="de">
    <div class="big bold">Regeste a</div>
    <br>
    <div class="paraatf">
      <span class="artref">Art. 51 Abs. 4 BGG</span>; Streitwert bei Rechtsöffnung.
      <div class="paratf">Künftige Unterhaltsbeiträge sind noch nicht fällig.</div>
    </div>
  </div>
  
  <div id="regeste" lang="de">
    <div class="big bold">Regeste b</div>
    <br>
    <div class="paraatf">
      <span class="artref">Art. 84 SchKG</span>; Kognition des Rechtsöffnungsgerichts.
      <div class="paratf">Die Rechtsöffnung ist zu verweigern.</div>
    </div>
  </div>
</body>
</html>
"""

soup = BeautifulSoup(test_html, "lxml")
regeste = parse_regeste(soup)

print("Extrahierte Regeste:")
print("=" * 50)
print(regeste)

---
# Teil 4: Die Hauptfunktion `main()`

Hier wird alles zusammengeführt:

In [ ]:
def main():
    """
    Hauptfunktion: Führt den gesamten Scraping-Prozess durch.
    """
    INDEX_URL = "https://search.bger.ch/ext/eurospider/live/de/php/clir/http/index_atf.php?year=151&volume=III&lang=de&zoom=&system=clir"
    out_csv = "bge_151_iii_regesten.csv"
    
    # Session erstellen (hält Verbindungen offen)
    with requests.Session() as session:
        
        # SCHRITT 1: Index-Seite laden
        print("Lade Index-Seite...")
        index_html = fetch_html(session, INDEX_URL)
        
        # SCHRITT 2: Links extrahieren
        decision_links = extract_decision_links(index_html, INDEX_URL)
        print(f"Gefunden: {len(decision_links)} Entscheide")
        
        if not decision_links:
            raise RuntimeError("Keine Entscheid-Links gefunden!")
        
        # SCHRITT 3: Jeden Entscheid besuchen
        rows = []
        for i, url in enumerate(decision_links, start=1):
            # Höflich sein: 0.5 Sekunden zwischen Anfragen warten
            time.sleep(0.5)
            
            # Seite laden und parsen
            html = fetch_html(session, url)
            soup = BeautifulSoup(html, "lxml")
            
            # Daten extrahieren
            urteilsnummer = parse_urteilsnummer(soup)
            regeste = parse_regeste(soup)
            
            # Warnungen ausgeben
            if not urteilsnummer:
                print(f"[WARN] Urteilsnummer nicht gefunden: {url}")
            if not regeste:
                print(f"[WARN] Regeste nicht gefunden: {url}")
            
            # Ergebnis speichern
            rows.append((urteilsnummer, regeste))
            print(f"[{i}/{len(decision_links)}] {urteilsnummer} extrahiert")
    
    # SCHRITT 4: CSV schreiben
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=";")  # Semikolon für CH-Excel
        writer.writerow(["urteilsnummer", "regeste"])  # Header
        writer.writerows(rows)  # Daten
    
    print(f"Fertig: {out_csv}")

print("Funktion main() definiert.")
print("")
print("Der Ablauf:")
print("1. Index-Seite laden")
print("2. Links zu allen Entscheiden extrahieren")
print("3. Jeden Entscheid besuchen und Daten extrahieren")
print("4. Alles in eine CSV-Datei schreiben")

### Warum `time.sleep(0.5)`?

**Höflichkeit!** Wenn wir zu schnell viele Anfragen senden, könnte:
- Die Website uns blockieren
- Der Server überlastet werden

Mit `time.sleep(0.5)` warten wir eine halbe Sekunde zwischen den Anfragen.

---
# Teil 5: Das alte Problem und die Lösung

## Was war das Problem mit dem ursprünglichen Code?

Der ursprüngliche Code suchte nach einem Element mit **exakt** dem Text `"Regeste"`:

In [ ]:
# ALTER CODE (funktionierte nicht bei allen Entscheiden):

def alte_methode(soup, label):
    """Sucht nach einem Element mit exakt diesem Text."""
    node = soup.find(string=lambda t: isinstance(t, str) and t.strip() == label)
    return node

# Das Problem: Bei manchen Entscheiden steht "Regeste a" statt "Regeste"

test_html = '<div class="big bold">Regeste a</div>'
soup = BeautifulSoup(test_html, "lxml")

gefunden = alte_methode(soup, "Regeste")
print(f"Suche nach 'Regeste': {gefunden}")

gefunden = alte_methode(soup, "Regeste a")
print(f"Suche nach 'Regeste a': {gefunden}")

## Die Lösung

Statt nach dem Text zu suchen, suchen wir nach der **HTML-Struktur**:

In [ ]:
# NEUE METHODE (funktioniert immer):

def neue_methode(soup):
    """Sucht nach allen <div id='regeste'> Elementen."""
    return soup.find_all("div", id="regeste")

test_html = """
<div id="regeste"><div class="big bold">Regeste a</div><div class="paraatf">Inhalt A</div></div>
<div id="regeste"><div class="big bold">Regeste b</div><div class="paraatf">Inhalt B</div></div>
"""

soup = BeautifulSoup(test_html, "lxml")
regesten = neue_methode(soup)

print(f"Gefunden: {len(regesten)} Regeste(n)")
for i, r in enumerate(regesten, 1):
    label = r.find("div", class_="big bold").get_text(strip=True)
    print(f"  {i}. {label}")

---
# Teil 6: Zusammenfassung

## Die wichtigsten Konzepte:

### 1. HTTP-Anfragen mit `requests`
```python
response = requests.get(url)
html = response.text
```

### 2. HTML parsen mit `BeautifulSoup`
```python
soup = BeautifulSoup(html, "lxml")
element = soup.find("div", id="regeste")
alle_elemente = soup.find_all("a", href=True)
text = element.get_text(strip=True)
```

### 3. Text suchen mit Regular Expressions
```python
import re
match = re.search(r"Muster", text)
neuer_text = re.sub(r"alt", "neu", text)
```

### 4. CSV schreiben
```python
with open("datei.csv", "w") as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerow(["spalte1", "spalte2"])
    writer.writerows(daten)
```

## Der Schlüssel zum Erfolg:

1. **HTML-Struktur analysieren** - Entwicklertools nutzen (F12)
2. **Muster erkennen** - Was haben alle Regesten gemeinsam?
3. **Gezielt suchen** - Nach Struktur, nicht nach Text
4. **Testen** - Mit verschiedenen Seiten prüfen

---
# Bonus: Den Scraper selbst ausführen

Du kannst den Scraper direkt hier ausführen (dauert ca. 30 Sekunden):

In [ ]:
# Entkommentiere die nächste Zeile, um den Scraper auszuführen:
# main()

---

**Gratulation!** Du hast jetzt einen Überblick, wie Web Scraping funktioniert.

Bei Fragen: Experimentiere mit dem Code, ändere Dinge, und schau was passiert!